# Three-step RAS computation for the $1s^2 2s^2\: ^1S_0$ and $1s^2 2s2p\: ^{1,3}P_1$ levels of neutral beryllium

In [1]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17539428043366158461\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15056505488447517363\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17539428043366158461\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


In JAC, restricted-active space (RAS) computations generally refer to systematically enlarged SCF computations, and which are often performed stepwise for additional layers of active orbitals. RAS computations are based on one several *reference configurations*, a well-defined *level symmetry* for all CSF bases as well as on virtual single- (S), double- (D), triple-excitations (T), etc. of electrons from valence orbitals $\to$ valence+virtual orbitals from the active set. These SDT+ excitations can be different for each steps, similar as the orbitals which are kept *frozen*. More often than not, another layer of active orbitals is added and optimized, while all orbitals from previous steps are kept frozen.

In this tutorial, we show the three-step RAS computations for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium as well as for its two lowest-excited $1s^2 2s2p\: ^{1,3}P_1$ levels. In these computations, we first compute the orbitals for $1s^2 2s^2$ reference configuration (in some configuration-average approach), and then stepwise enlarge the active orbitals by $2s,\: 2p$ (step$_1$), $3s,\: 3p,\: 3d$ (step$_2$), and finally $4s,\: ...,\: 4f$ (step$_3$).
In all these steps, the $1s$ orbitals as well as the orbitals from the previous steps are kept frozen.

Before we shall explain further details about such RAS computations, however, let us however start with the `struct Atomic.RasComputation`, which is used to *describe* a complete (RAS) calculations in all detail and which can then be *perform*(ed) as usual:

In [2]:
? Atomic.RasComputation

`struct  Atomic.RasComputation`       ... a struct for defining a series of restricted active space computations for a specified set of levels.          This type provides all information to generate the atomic basis and to perform the corresponding SCF and          multiplet computations for a specified number of levels.

```
+ name             ::String                    ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model             ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid               ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}    ... List of references configurations, at least 1.
+ symmetry         ::LevelSymmetry             ... Symmetry of the levels/CSF in the many-electron basis.
+ NoElectrons      ::Int64                     ... Number of electrons.
+ steps            ::Array{Atomic.RasStep,1}   ... List of SCF steps that are to be done in this model computation.
+ settings         ::Atomic.RasSettings        ... Settings for the given RAS computation
```

---

`JAC.Atomic.RasComputation()`  ... constructor for an 'empty' instance of the a variable::Atomic.RasComputation


Apart from the nuclear model, we need here to provide a list of (non-relativistic) reference configurations, a single level symmetry as well as a **list of steps,** which specify the virtual excitations and which of the orbitals are kept frozen during this particular step:

In [3]:
? RasStep

search: RasStep RasSettings uppercasefirst lowercasefirst



`struct  Atomic.RasStep`       ... specifies an individual step of a (relativistic) restricted active space computation for a set of levels. This struct          comprises all information to generate the orbital basis and to perform the associated SCF and multiplet computations for a          selected number of levels.

```
+ seFrom            ::Array{Shell,1}        ... Single-excitations from shells   [sh_1, sh_2, ...]
+ seTo              ::Array{Shell,1}        ... Single-excitations to shells  [sh_1, sh_2, ...]
+ deFrom            ::Array{Shell,1}        ... Double-excitations from shells   [sh_1, sh_2, ...]
+ deTo              ::Array{Shell,1}        ... Double-excitations to shells  [sh_1, sh_2, ...]
+ teFrom            ::Array{Shell,1}        ... Triple-excitations from shells   [sh_1, sh_2, ...]
+ teTo              ::Array{Shell,1}        ... Triple-excitations to shells  [sh_1, sh_2, ...]
+ qeFrom            ::Array{Shell,1}        ... Quadrupole-excitations from shells   [sh_1, sh_2, ...]
+ qeTo              ::Array{Shell,1}        ... Quadrupole-excitations to shells  [sh_1, sh_2, ...]
+ frozenShells      ::Array{Shell,1}        ... List of shells that are kept 'frozen' in this step.
+ constraints       ::Array{String,1}       ... List of Strings to define 'constraints/restrictions' to the generated CSF basis.
```

---

`JAC.Atomic.RasStep()`  ... constructor for an 'empty' instance of a variable::Atomic.RasStep

---

`JAC.Atomic.RasStep(rasStep::Atomic.RasStep;`

```
                seFrom::Array{Shell,1}=Shell[], seTo::Array{Shell,1}=Shell[], 
                deFrom::Array{Shell,1}=Shell[], deTo::Array{Shell,1}=Shell[], 
                teFrom::Array{Shell,1}=Shell[], teTo::Array{Shell,1}=Shell[], 
                qeFrom::Array{Shell,1}=Shell[], qeTo::Array{Shell,1}=Shell[], 
                frozen::Array{Shell,1}=Shell[], constraints::Array{String,1}=String[]  
... constructor for modifying the given rasStep by specifying all excitations, frozen shells and constraints optionally.
```


In particular the last constructor enables one to *overwrite* either the default constructor `RasStep()` or, often more useful, some previous step. **We note, however, that presently no constraints are taken into account in all the RAS computations.** Moreover, let us consider here also the *settings* for such RAS computations which apply (equally) to *all* steps:

In [4]:
? RasSettings

search: RasSettings



`struct  Atomic.RasSettings`       ... a struct for defining the settings for a restricted active-space computations.

```
+ levelsScf            ::Array{Int64,1}     ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64              ... maximum number of SCF iterations in each RAS step.
+ accuracyScf          ::Float64            ... convergence criterion for the SCF field.

+ breitCI              ::Bool               ... logical flag to include Breit interactions.
+ selectLevelsCI       ::Bool               ... true, if specific level (number)s have been selected.
+ selectedLevelsCI     ::Array{Int64,1}     ... Level number that have been selected.
```

---

`Atomic.RasSettings()`  ... constructor for setting the default values.


These settings enables one to specify the levels on which the SCF is optimized as well as the maximum number of iterations and requested accuracy in this SCF process. In addition, the flag `breitCI` determines the use of the Breit interaction in the set-up (and diagonalization) of the Hamiltonian matrix; this flag does not affect the *next* RAS step but may incluence further computations if the multiplets of the individual steps are subsequently used in computations.

In [6]:
Atomic.RasSettings()

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       false  
selectedLevelsCI:     Int64[]  


In [7]:
rasSettings = RasSettings([1], 24, 1.0e-6, false, true, [1,2,3] )

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       true  
selectedLevelsCI:     [1, 2, 3]  


The last two entries to these settings tell JAC that we wish to compute (and print) always the lowest three levels of each Hamiltonian matrix; **this feature is however unused at present since, until now, the whole Hamiltonian matrix is diagonalized (and printed) in all cases.**

We are now prepared to define the RAS computation as a whole, i.e. a name (mainly for the printout), the reference configuration(s) as well as in turn all (three) steps of a RAS computation for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium:

In [8]:
name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs  = [Configuration("[He] 2s^2")]

1-element Array{Configuration,1}:
 Configuration: 1s^2 2s^2 

We also specify that we will keep the $1s$ orbital frozen in the computation of all orbitals beyond those in the reference configurations as well as that all excitations are to be taken from the $2s$ shell:

In [9]:
frozen      = [Shell("1s")]
from        = [Shell("2s")]

1-element Array{Shell,1}:
 2s

In [10]:
to          = [Shell("2s"), Shell("2p")]
step1       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))


RAS step with 1 frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }


Similarly, we can specify the second and third step by using (and overwriting) the information provided for the previous step; the use of deepcopy() ensures that no internal calls are repeated if these cells are called in different order:

In [11]:
append!(frozen, [Shell("2s"), Shell("2p")])
append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


RAS step with 3 frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }


In [12]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


RAS step with 6 frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }


With these definition of all *steps*, we can now define the RAS computations as a whole and subsequently *run* it:

In [13]:
wa          = RasComputation(name, Nuclear.Model(4.), Radial.Grid("grid: exponential"), refConfigs, LevelSymmetry(0, Basics.plus),
                             4, [step1, step2, step3], rasSettings)

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines with break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


RAS computation: 'Beryllium 1s^2 2s^2 ^1S_0 ground state' for symmétry 0 + and with 3 steps as well as with reference configurations: 
   1s^2 2s^2 ,  
and the current settings:
NoElectrons:          4  
steps:                RasStep[
RAS step with 1 frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
RAS step with 3 frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
RAS step with 6 frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
settings:             levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breit

In [14]:
wb = perform(wa, output=true)


... in perform['computation: SCF'] ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27214942e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.70620128e-07    
      3      3s_1/2      -8.89087769e-01    -8.89078304e-01    -1.06460554e-05    
      4      4s_1/2      -5.00168012e-01    -5.0008

Dict{String,Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

Of course, analogue computations can be readily carried out also for the (two) $1s^2 2s2p\: ^{1,3}P_1$ excited levels of neutral beryllium, and where we use major parts of the previous computation:

In [15]:
name2        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs2  = [Configuration("[He] 2s^2")]
wa2          = RasComputation(name2, Nuclear.Model(4.), Radial.Grid("grid: exponential"), refConfigs2, LevelSymmetry(1, Basics.minus),
                              4, [step1, step2, step3], rasSettings)

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines with break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


RAS computation: 'Beryllium 1s^2 2s^2 ^1S_0 ground state' for symmétry 1 - and with 3 steps as well as with reference configurations: 
   1s^2 2s^2 ,  
and the current settings:
NoElectrons:          4  
steps:                RasStep[
RAS step with 1 frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
RAS step with 3 frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
RAS step with 6 frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
settings:             levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breit

In [16]:
wb2 = perform(wa2, output=true)


... in perform['computation: SCF'] ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27214942e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.70620128e-07    
      3      3s_1/2      -8.89087769e-01    -8.89078304e-01    -1.06460554e-05    
      4      4s_1/2      -5.00168012e-01    -5.0008

Dict{String,Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

All *results* of this and the previous RAS computations are kept by the dictionaries `wb2` and `wb` above. We can use these variables to extract, for instance, the multiplets as obtained from the first step:

In [18]:
wm = wb["reference multiplet"];   wm2 = wb2["reference multiplet"]

name:        Reference multiplet:  
levels:      Level[Level: J = 0, M = 0, parity = plus, index = 1 
energy:         -14.570469412671915  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [1.0]  
]  


In [ ]:
Obviously, the describe the same *level* since they are both built just open the $1s^2 2s^2$ closed-shell configuration:

In [21]:
println("Energies from references conf:  $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

Energies from references conf:  -14.570469412671915   -14.570469412671915   


These energies differ, however, if we compare them for different steps as well as for the two different RAS computations.

In [23]:
wm = wb["step1"];   wm2 = wb2["step1"]
println("Energies from step 1:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

Energies from step 1:           -14.605107808213766   -14.503651075293673   


In [24]:
wm = wb["step2"];   wm2 = wb2["step2"]
println("Energies from step 2:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

Energies from step 2:           -14.615971325241643   -14.518566797622723   


In [25]:
wm = wb["step3"];   wm2 = wb2["step3"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

Energies from step 3:           -14.616791191361385   -14.519297960852532   
